# 스크래핑, API 연습

## 1. 스크래핑 - requests, beautifulsoup

In [2]:
# requests
import requests
from bs4 import BeautifulSoup

In [9]:
# 가져올 웹페이지의 URL
url = 'https://news.google.com/search?q=%EC%9E%A5%EC%9A%B0%EC%98%81&hl=ko&gl=KR&ceid=KR%3Ako'

# HTTP GET 요청을 보내고 응답받기
response = requests.get(url)

# 응답이 성공적으로 왔는지 확인
if response.status_code == 200:
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser') # response.content: 이미지, 파일일 경우!
    # <title> 태그 자체 찾기(태그+내용 전체 포함)
    title_tag = soup.title
    # <title> 태그의 텍스트 출력(태그는 빼고 내용만 깔끔하게 추출)
    print('페이지 제목:', title_tag.string)
else:
    print('웹페이지를 불러오는 데 실패했습니다.')

# data = soup.select('.product-price')

페이지 제목: Google 뉴스 - 검색


#### BeautifulSoup 함수
- soup.prettify(): HTML, XML 문서 내용 보기 좋게 정렬
- soup.find('div'): 조건에 맞는 태그 하나
- soup.find_all('div'): 조건에 맞는 태그 여러 개
- soup.select('.item > a'): CSS 선택자로 태그 찾기 여러 개
- soup.select_one('#main'): CSS 선택자로 하나
- tag.string: 텍스트 하나만 있을 때
- tag.get_text(): 내부 모든 텍스트 추출, 태그 안의 텍스트 내용을 가져옴 ex) 구글
- link.get('href'): 속성(attribute) 값을 가져옴 ex) https://google.com

#### find/select 차이?
#### find
- find / find_all 은 HTML 태그 이름 + 속성 위주 -> 태그 자체를 기준으로 찾는 방식!
- soup.find('div', class_='price') / soup.find_all('a', href=True) -> 구조가 단순하거나, 특정 태그 + 클래스명이 확실할 때!

#### select
- select / select_one 은 CSS 선택자 기반 -> 웹페이지 구조가 복잡하거나, 특정 위치/계층을 지정해서 찾을 때
- soup.select('.price > span strong') / soup.select_one('#main-product .info a')

#### CSS 선택자?
- 웹페이지에서 특정 태그(요소)를 고르는 규칙, CSS(스타일 줄 때) 문법을 BeautifulSoup도 사용 -> 웹페이지 꾸밀 때 쓰는 방법 = 크롤링할 때 요소 찾는 방법
- 계층(부모 → 자식) 선택

| 선택자 형태   | 의미          | 예시                 |
| -------- | ----------- | ------------------ |
| `.class` | 클래스 이름으로 선택 | `.price`           |
| `#id`    | 아이디로 선택     | `#main-product`    |
| `tag`    | 태그 이름으로 선택  | `span`, `a`, `div` |

| 문법      | 읽는 방법        | 의미                      |
| ------- | ------------ | ----------------------- |
| `A B`   | A 안에 있는 B    | (A 안에 B가 *어디든* 들어있으면 됨) |
| `A > B` | A 바로 아래 자식 B | (중간에 다른 태그가 끼면 안됨)      |

- soup.select('.price > span strong'): price 클래스 안 바로 아래 있는 span 태그 그 안에 있는 strong 태그들 모두 찾기 (주로 가격)
- soup.select_one('#main-product .info a'): id=main-product 블록 안의 info 클래스에서 링크(a 태그) 하나 가져오기 (주로 제품정보 링크)

In [18]:
# <p>Hello <b>World</b>!</p>

# tag = soup.find('p')
# print(tag.string)      # ❌ None 나올 수 있음
# print(tag.get_text())  # ✅ "Hello World!"

# tag = soup.find('div')           # 첫 번째 div
# tag = soup.find('span', class_='price')  # class가 price인 span 하나

# <a href="https://google.com">구글</a>

# tags = soup.find_all('a')   # 모든 a
# for t in tags:
#     print(t.get_text()) # 태그 안 텍스트 내용 가져옴 ex) 구글

# links = soup.find_all('a')
# for link in links :
#     print(link.get('href')) # 속성 값만 가져옴 ex) https://google.com

# soup.select('div.price')        # <div class="price">...</div>
# soup.select('#main > ul li')    # id=main인 요소 아래 ul 안의 모든 li
# soup.select('a[href]')          # href 속성이 있는 a 태그
# soup.select('.product-price')   # CSS 선택자를 이용해서 HTML 안에서 class="product-price" 인 태그들을 전부 찾아 리스트로 가져오는 코드
                                  # . 은 class 선택자를 의미

# soup.select_one('div.price')   # 하나만

## 2. 스크래핑 - selenium

In [10]:
# 크롬 드라이버 경로 안 잡고 webdriver_manager 사용하면 자동으로 설치!

!pip install webdriver-manager

import time
import pandas as pd

from selenium import webdriver

from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 웹 브라우저 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹 페이지 열기
driver.get("https://www.google.com")

# 잠시 대기 (페이지 로딩 대기용)
time.sleep(2)

# <title> 태그 가져오기
title = driver.title
print('페이지 제목:', title)

# 웹 브라우저 닫기
driver.quit()


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
페이지 제목: Google


In [15]:
# 웹브라우저 드라이버 설치하기

# 셀레늄은 브라우저를 자동으로 조작하는 라이브러리라서
# 브라우저에 맞는 드라이버가 필요해.

# 크롬 브라우저 쓰고 있다면:

# 크롬 버전 확인 → 설정 → 도움말 → Chrome 정보
# 버전 예: 120.x.xxxx

# 그다음 여기에 맞는 ChromeDriver 다운로드:
# https://googlechromelabs.github.io/chrome-for-testing/

# 다운로드 후 압축 풀고 chromedriver.exe 위치 기억하기.

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service

# service = Service("chromedriver.exe")  # 다운로드한 chromedriver 경로 위치 넣기
# driver = webdriver.Chrome(service=service)

# driver.get("https://www.google.com")

# print(driver.title)

# driver.quit()

In [11]:
# 판다스, 셀레늄 불러오기

import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

In [16]:
# 키워드 설정 및 URL 작성
KEYWORD = '장우영'
URL = f"https://news.google.com/search?q={KEYWORD}%20when%3A7d&hl=ko&gl=KR&ceid=KR%3Ako"

# 크롬 실행
driver = webdriver.Chrome()

# URL 열기
driver.get(URL)

# <title> 태그 가져오기
title = driver.title
print('페이지 제목:', title)

# 잠시 대기
time.sleep(2)

페이지 제목: Google 뉴스 - 검색


#### Selenium 함수
- driver.find_element(By.CLASS_NAME, '') * By.ID / By.CSS_SELECTOR / By.NAME / By.TAG_NAME / By. XPATH / By.LINK_TEXT / By.PARTIAL_LINK_TEXT
- driver.find_elements(By.TAG_NAME, '')
- search_button.click(): 마우스 클릭
- search_box.send_keys('Selenium'): 텍스트(Selenium) 입력
- link.get_attribute('href'): 속성 중 하나의 값을 가져옴 (= get('href'))
- driver.find_element(By.TAG_NAME, "a").text: 태그 안의 텍스트 내용을 가져옴 (= get_text())
- driver.back() / driver.forward() / driver.refresh()
- WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "my-element")))

In [22]:
# search_button = driver.find_element(By.NAME, 'btnK')
# seach_button.click()

# search_box = driver.find_element(By.NAME, 'q')
# search_box.send_keys('Selenium')

# link = driver.find_element(By.TAG_NAME, "a")
# print(link.get_attribute("href"))

# text = driver.find_element(By.TAG_NAME, "a").text
# print(text)

# 뒤로가기, 앞으로가기, 새로고침
# driver.get("https://example.com")     # A 페이지
# driver.get("https://google.com")      # B 페이지

# driver.back()     # 뒤로가기 (B → A)
# driver.forward()  # 앞으로가기 (A → B)
# driver.refresh()  # 새로고침

# 웹 페이지 완전히 로드 or 특정한 조건 충족될 때까지 코드 실행 잠시 멈추는 기능
# 1. 암묵적 대기(implicit wait)
# driver.implicitly_wait(10)

# 2. 명시적 대기(explicit wait)
# 1) 요소 한 개 찾을 때까지 기다리기
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By

# driver.get("https://example.com")

# element = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "product"))  # EC = Expected Conditions (기다릴 조건) / presence_of_element_located: 해당 요소가 HTML 안에 존재하기만 하면 OK
# )

# print(element.text)

# 2) 요소 전부 찾을 떄까지 기다리기
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By

# elements = WebDriverWait(driver, 10).until(
#     EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product"))
# )

# for e in elements:
#     print(e.text)

# 3. 유연한 대기(fluent wait)
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

# 10초 동안, 2초마다 재시도, 특정 예외는 무시
# wait = WebDriverWait(
#     driver,
#     timeout=10,
#     poll_frequency=2,
#     ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)
# )

# 1) 람다로 직접 조건 지정 (Java FluentWait 스타일과 가장 유사)
# element = wait.until(lambda d: d.find_element(By.ID, "myElement"))

# 2) EC(기대 조건) 사용
# element = wait.until(EC.presence_of_element_located((By.ID, "myElement")))

In [17]:
# 기사 요소 전부 가져오기
# data = driver.find_elements(By.CLASS_NAME, 'dynamic-content')
data = driver.find_elements(By.TAG_NAME, 'article')

# 빈 리스트 만들기
data_list = []

# 클래스 이름으로 기사 제목, 날짜, 언론사 수집하기
for i in data :
    title = i.find_element(By.CLASS_NAME, 'JtKRv').text
    date = i.find_element(By.CLASS_NAME, 'hvbAAd').text
    press = i.find_element(By.CLASS_NAME, 'MCAGUe').text

    # 빈 리스트에 기사 제목, 날짜, 언론사 내용 추가하기
    data_list.append({
        '기사 제목' : title,
        '날짜' : date,
        '언론사' : press
    })

# 크롬 종료
driver.quit()

In [11]:
# 리스트 내용 확인
data_list

[{'기사 제목': '경남장애인수영연맹 장우영, 장애인체전 2년 연속 ‘5관왕’', '날짜': '6일 전', '언론사': '경남일보'},
 {'기사 제목': '[장애인체전] 경남 목표했던 종합 9위로 마무리', '날짜': '5일 전', '언론사': '경남도민일보'},
 {'기사 제목': '2PM 장우영, 7년 5개월 만에 컴백…초여름 뜨겁게 달군다', '날짜': '4일 전', '언론사': 'MSN'},
 {'기사 제목': '치과에 피부과 의사까지 …연하남 직업 공개 후 러브라인 변화?(누난내게여자야) - 조선비즈',
  '날짜': '3시간 전',
  '언론사': 'Chosun Biz'},
 {'기사 제목': '한혜진 충격…손예진·조승우의 \'클래식\' 모른단 수빈에 "마상" (\'누내여\') [종합]',
  '날짜': '7일 전',
  '언론사': '네이트'},
 {'기사 제목': '\'장관급 인사\' 박진영, 53년 인생 첫 경험…"내손내잡 로망" (\'푹쉬면 다행이야\')',
  '날짜': '어제',
  '언론사': 'v.daum.net'},
 {'기사 제목': "김지석, ♥이주명 앞날 안 막았다…'신입사원 강회장' 출연 확정 [공식]",
  '날짜': '3일 전',
  '언론사': 'MSN'},
 {'기사 제목': '한혜진, 새 연하남에 \'태평양 어깨\'에 흥분 "어깨가 앞뒤로 두껍다…귀여워" (누난 내게)',
  '날짜': '어제',
  '언론사': '네이트'},
 {'기사 제목': '연하남 직진 로맨스 그렸는데…1.3→0.6%로 시청률 반토막 나버린 작품',
  '날짜': '3시간 전',
  '언론사': '네이트'},
 {'기사 제목': '\'라디오쇼\' 곽튜브, 상상 못한 축의금 받은 결혼식…"열심히 살았나 싶다" 감격 심경 [순간포착]',
  '날짜': '10시간 전',
  '언론사': 'MSN'},
 {'기사 제목': '지석진 "송지효 매주 봐서 예쁜지 몰라…시상식 때 매번 사과한다" (\'지효쏭\')[순간포착]',
  '날짜': '어제',
 

In [12]:
# Pandas DataFrame으로 정리
df = pd.DataFrame(data_list)
df.head()

,기사 제목,날짜,언론사
0,"경남장애인수영연맹 장우영, 장애인체전 2년 연속 ‘5관왕’",6일 전,경남일보
1,[장애인체전] 경남 목표했던 종합 9위로 마무리,5일 전,경남도민일보
2,"2PM 장우영, 7년 5개월 만에 컴백…초여름 뜨겁게 달군다",4일 전,MSN
3,치과에 피부과 의사까지 …연하남 직업 공개 후 러브라인 변화?(누난내게여자야) - ...,3시간 전,Chosun Biz
4,"한혜진 충격…손예진·조승우의 '클래식' 모른단 수빈에 ""마상"" ('누내여') [종합]",7일 전,네이트


In [13]:
# CSV 파일로 저장
df.to_csv('구글뉴스_장우영_sub.csv', index=False, encoding='utf-8-sig')

## 3. API - requests

In [18]:
# 공공 API 선택: 서울열린데이터광장 API

# API 인증키 발급 과정
# [1] 인증키 신청: https://data.seoul.go.kr/together/mypage/actkeyMain.do
#  - 서비스 환경(연구), 사용 URL(http://openapi.seoul.go.kr:8088), 이메일, 활용용도(학술), 내용(데이터 분석 학습) 입력

# [2] 인증키 발급: 576*****************(가명처리)

# [3] 인증키 관리: API명(서울시 지하철 호선별 역별 승하차 인원 정보)
#  - 사용일자(20251107), 호선(선택x), 역명(선택x)
#  - 요청주소 확인(http://openapi.seoul.go.kr:8088/sample/xml/CardSubwayStatsNew/1/500/20251016/%20/%20/)

In [19]:
# requests 라이브러리를 사용한 데이터 수집 코드 작성(최소 100건 이상)
# requests 불러오기
import requests

# url 작성
url = 'http://openapi.seoul.go.kr:8088/576b424544776c6436366961486d59/json/CardSubwayStatsNew/1/500/20251107/%20/%20/'

response = requests.get(url)
print(response.content)

b'{"CardSubwayStatsNew":{"list_total_count":617,"RESULT":{"CODE":"INFO-000","MESSAGE":"\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4"},"row":[{"USE_YMD":"20251107","SBWY_ROUT_LN_NM":"1\xed\x98\xb8\xec\x84\xa0","SBWY_STNS_NM":"\xec\x84\x9c\xec\x9a\xb8\xec\x97\xad","GTON_TNOPE":"91746","GTOFF_TNOPE":"88729","REG_YMD":"20251110"},{"USE_YMD":"20251107","SBWY_ROUT_LN_NM":"1\xed\x98\xb8\xec\x84\xa0","SBWY_STNS_NM":"\xec\x8b\x9c\xec\xb2\xad","GTON_TNOPE":"33474","GTOFF_TNOPE":"34109","REG_YMD":"20251110"},{"USE_YMD":"20251107","SBWY_ROUT_LN_NM":"1\xed\x98\xb8\xec\x84\xa0","SBWY_STNS_NM":"\xec\xa2\x85\xea\xb0\x81","GTON_TNOPE":"47035","GTOFF_TNOPE":"46054","REG_YMD":"20251110"},{"USE_YMD":"20251107","SBWY_ROUT_LN_NM":"1\xed\x98\xb8\xec\x84\xa0","SBWY_STNS_NM":"\xec\xa2\x85\xeb\xa1\x9c3\xea\xb0\x80","GTON_TNOPE":"31202","GTOFF_TNOPE":"28183","REG_YMD":"20251110"},{"USE_YMD":"20251107","SBWY_ROUT_LN_NM":"1\xed\x98\xb8\xec\x84\xa0","

In [20]:
# 1. Extract: API로부터 JSON/XML 데이터 추출
# json 불러오기
import json

# 한글 표시되도록 하기
response.encoding = 'utf-8'

# json 파이썬 객체로 변환
result = json.loads(response.text)

# json 읽기 편하게 출력(들여쓰기, 다 영어는 아님)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "CardSubwayStatsNew": {
    "list_total_count": 617,
    "RESULT": {
      "CODE": "INFO-000",
      "MESSAGE": "정상 처리되었습니다"
    },
    "row": [
      {
        "USE_YMD": "20251107",
        "SBWY_ROUT_LN_NM": "1호선",
        "SBWY_STNS_NM": "서울역",
        "GTON_TNOPE": "91746",
        "GTOFF_TNOPE": "88729",
        "REG_YMD": "20251110"
      },
      {
        "USE_YMD": "20251107",
        "SBWY_ROUT_LN_NM": "1호선",
        "SBWY_STNS_NM": "시청",
        "GTON_TNOPE": "33474",
        "GTOFF_TNOPE": "34109",
        "REG_YMD": "20251110"
      },
      {
        "USE_YMD": "20251107",
        "SBWY_ROUT_LN_NM": "1호선",
        "SBWY_STNS_NM": "종각",
        "GTON_TNOPE": "47035",
        "GTOFF_TNOPE": "46054",
        "REG_YMD": "20251110"
      },
      {
        "USE_YMD": "20251107",
        "SBWY_ROUT_LN_NM": "1호선",
        "SBWY_STNS_NM": "종로3가",
        "GTON_TNOPE": "31202",
        "GTOFF_TNOPE": "28183",
        "REG_YMD": "20251110"
      },
      {
        "USE_YMD": "

In [21]:
# 2. Transform: 필요한 필드만 선택, 데이터 타입 변환
# 3. Load: 정제된 데이터를 Pandas DataFrame으로 적재 후 CSV 저장
# 판다스 불러오기
import pandas as pd

# 1) 필요한 필드만 선택
items = result['CardSubwayStatsNew']['row']

# 2) 판다스 데이터프레임으로 변환
df = pd.DataFrame(items)

# 3) 데이터 타입 변환
df['GTON_TNOPE'] = df['GTON_TNOPE'].astype(int)
df['GTOFF_TNOPE'] = df['GTOFF_TNOPE'].astype(int)
print(df.dtypes)

# 4) CSV 저장
df.to_csv('서울지하철호선별역별승하차인원_20251107.csv', index=False, encoding='utf-8-sig')

USE_YMD            object
SBWY_ROUT_LN_NM    object
SBWY_STNS_NM       object
GTON_TNOPE          int64
GTOFF_TNOPE         int64
REG_YMD            object
dtype: object


In [22]:
# matplotlib 불러오기
import matplotlib.pyplot as plt

# 한글 깨짐 방지
from matplotlib import rcParams

# Mac: 애플고딕
rcParams['font.family'] = 'AppleGothic'